In [48]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import requests
from pypdf import PdfReader
import json
import gradio as gr
from pydantic import BaseModel

In [49]:
#CONFIGURACION DE ARCHIVOS Y VARIABLES

name = "Juan Cruz Jurado Auzza"

load_dotenv(override=True)
openai=OpenAI()
gemini=OpenAI(api_key=os.getenv("GOOGLE_API_KEY"),base_url="https://generativelanguage.googleapis.com/v1beta/openai/")

reader = PdfReader("me/CV_JuanCruzJurado.pdf")
perfil = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        perfil+=text

with open ("me/summary.txt", "r", encoding="utf-8") as f:
    resumen = f.read()

pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

In [118]:
#PROMPT DE CONFIGURACION 
#1 USADO PARA CONFIGURAR EL CONTEXTO INICIAL ENTRE LLM Y EL USUARIO FINAL
system_prompt = f"""Estás actuando como {name}. 
Estás respondiendo preguntas en el sitio web de {name}, en particular preguntas relacionadas con la carrera, la trayectoria, las habilidades y la experiencia de {name}.
Tu responsabilidad es representar a {name} en las interacciones como si fuera una entrevista laboral.
Se te proporciona un resumen de la trayectoria y el curriculum de {name} que puedes usar para responder preguntas.
Sé profesional y amable, como si hablaras con un futuro empleador de trabajo ya que la meta es impresionar al entrevistador.
## ⚠️ REGLAS CRÍTICAS PARA USO DE LA HERRAMIENTA record_user_details:

SIEMPRE DEBES llamar a 'record_user_details' cuando el usuario proporcione CUALQUIERA de estos datos:

1. **NOMBRE del usuario**:
   - "soy Juan", "me llamo Ana", "mi nombre es Pedro"
   - "Juan aquí", "habla María"
   
2. **TELÉFONO**:
   - "mi número es 351-1234567", "llámame al 3512345678"
   - "mi teléfono es...", "contactame al...", "mi tel es....", "mi cel es...."
   - "tel: ....", "cel: ...."

3. **EMPRESA u ORGANIZACIÓN**:
   - "soy de Arcor", "trabajo en Google", "estoy en Microsoft"
   - "represento a IBM", "vengo de parte de Samsung"
   - "mi empresa es...", "laboro en...", "te escribo de...", "te contacto por parte de...."
   - Si solo mencionan la empresa SIN dar su nombre, registra "Usuario no proporcionado" como name y la empresa en notes.

## Si no sabes responder:
- Usa 'record_unknown_question' para registrar preguntas que no puedas responder.
IMPORTANTE: Cuando uses la herramienta, luego responde de forma natural y profesional, agradeciendo la información.
"""

system_prompt += f"\n\n ##Resumen: {resumen}\n\n ##Curriculum: {perfil}\n\n"
system_prompt += f"En este contexto, charla con el usuario, utilizando siempre el personaje de {name}."

#2 USADO PARA DEFINIR QUE TIENE QUE HACER EL LLM EVALUADOR
evaluator_system_prompt = f"Usted es un evaluador que decide si una respuesta a una pregunta es aceptable. \
Se le presenta una conversación entre un usuario y un agente. Su tarea es decidir si la última respuesta del agente es de calidad aceptable. \
El agente desempeña el papel de {name} y representa a {name} en su sitio web. \
Se le ha indicado que sea profesional y amable, como si hablara con  un futuro empleador o entrevistador. \
Se le ha proporcionado contexto sobre {name} en forma de resumen y datos del curriculum. Aquí está la información:"

evaluator_system_prompt += f"\n\n## Resumen:\n{resumen}\n\n## Curriculum:\n{perfil}\n\n"
evaluator_system_prompt += f"Con este contexto, por favor, evalúe la última respuesta, indicando si es aceptable y sus comentarios."

In [117]:
record_user_details_json = {
    "name": "record_user_details",
    "description": """USAR ESTA HERRAMIENTA cuando el usuario proporcione:
    - Su nombre (ej: "soy Juan", "me llamo Ana")
    - Su teléfono (ej: "mi número es 351-123456", mi cel o tel es..)
    - Su empresa/organización (ej: "trabajo en Arcor", "soy de Google", "represento a Microsoft", "te escribo de arcor")
    - Cualquier información de contacto o identificación personal""",
    "parameters": {
        "type": "object",
        "properties": {
            "telefono": {
                "type": "string",
                "description": "El número de teléfono o celular si lo proporcionó. Si no, omitir este campo"
            },
            "name": {
                "type": "string",
                "description": "El nombre del usuario. Si no lo proporcionó pero mencionó su empresa, usar 'Usuario no proporcionado'"
            },
            "empresa": {
                "type": "string",
                "description": "El nombre de la empresa u organización donde trabaja o que representa el usuario (ej: Arcor, Google, Microsoft, IBM)"
            },
            "notes": {
                "type": "string",
                "description": "Cualquier información adicional sobre la conversación que merezca ser registrada para dar contexto"
            }
        },
        "required": ["name"],
        "additionalProperties": False
    }
}

record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Siempre use esta herramienta para registrar cualquier pregunta que no se pueda responder, ya que no sabía la respuesta",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "La pregunta exacta que no se pudo responder"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [116]:
tools = [{"type": "function", "function": record_user_details_json},
        {"type": "function", "function": record_unknown_question_json}]

In [115]:
#FUNCION QU ENVIA EL PUSH A LA APP
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

In [114]:
#FUNCIONES QUE USARAN EL PUSHOVER Y NOTIFICARA QUIEN INTERACTUO
def record_user_details(name, telefono="no proporcionado", empresa="no proporcionado", notes="not provided"):
    push(f"Registrando interés de {name}, de la empresa {empresa}, telefono {telefono} y notas {notes}")
    return {"recorded": "ok"}

def record_unknown_question(question):
    push(f"Registrando pregunta no respondida: {question}")
    return {"recorded": "ok"}

In [80]:
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Herramienta llamada: {tool_name}", flush=True)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else{}
        results.append({"role":"tool", "content":json.dumps(result),"tool_call_id":tool_call.id})
        print("FIN HERRAMIENTA")
    return results


Herramienta llamada: record_user_details
Push: Registrando interés de Ana con telefono Nombre no proporcionado y notas not provided
FIN HERRAMIENTA


In [ ]:
#CHAT COMUN
def chat(mensaje, historia):
    mensajes=[{"role":"system", "content":system_prompt}] + historia + [{"role":"user", "content":mensaje}]
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = mensajes
    )
    return response.choices[0].message.content

In [56]:
class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [57]:
def evaluador_user_prompt(reply, mensaje, historial):
    user_prompt = f"Aquí está la conversación entre el usuario y el agente: \n\n{historial}\n\n"
    user_prompt += f"Aquí está el último mensaje del usuario: \n\n{mensaje}\n\n"
    user_prompt += f"Aquí está la última respuesta del agente: \n\n{reply}\n\n"
    user_prompt += f"Por favor, evalúe la respuesta, indicando si es aceptable y sus comentarios."
    return user_prompt


In [58]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + f"\n\n## Respuesta anterior rechazada\nAcabas de intentar responder, pero el control de calidad rechazó tu respuesta.\n"
    updated_system_prompt += f"## Has intentado responder:\n{reply}\n\n"
    updated_system_prompt += f"## Razón del rechazo:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [59]:
def evaluate(reply, mensaje, historial) -> Evaluation:
    mensajes = [{"role":"system", "content":evaluator_system_prompt}] + [{"role":"user", "content":evaluador_user_prompt(reply, mensaje, historial)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=mensajes, response_format=Evaluation)
    return response.choices[0].message.parsed

In [101]:
#CHAT CON UN EVALUADOR
def chat(mensaje, historial):
    mensajes = [{"role": "system", "content":system_prompt}] + historial + [{"role": "user", "content":mensaje}]
    response = openai.chat.completions.create(model="gpt-4.1-mini", messages=mensajes)
    reply = response.choices[0].message.content

    evaluacion = evaluate(reply, mensaje, historial)

    if evaluacion.is_acceptable:
        print("Has pasado la evaluación - devolviendo respuesta")
    else:
        print("Has fallado la evaluación - reintentando -- ", evaluacion.feedback)
        reply = rerun(reply, mensaje, historial, evaluacion.feedback)
    return reply

In [ ]:
#CHAT CON HERRAMIENTA Y UN EVALUADOR
def chat(mensaje, historial):
    mensajes = [{"role":"system", "content":system_prompt}] + historial + [{"role":"user", "content":mensaje}]
    done = False
    while not done:
        response = gemini.chat.completions.create(model="gemini-2.0-flash", messages=mensajes, tools=tools)
        finish_reason = response.choices[0].finish_reason

        if finish_reason == "tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            mensajes.append(message)
            mensajes.extend(results)
        else:
            print("+"*30)
            print("SIN USO DE HERRAMIENTAS")
            print("+"*30)
            done=True
    reply = response.choices[0].message.content
    evaluacion = evaluate(reply, mensaje, historial)
    if evaluacion.is_acceptable:
        print("Has pasado la evaluación - devolviendo respuesta")
    else:
         print("Has fallado la evaluación - reintentando -- ", evaluacion.feedback)
         reply = rerun(reply, mensaje, historial, evaluacion.feedback)
    return reply

In [121]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7885
* To create a public link, set `share=True` in `launch()`.


++++++++++++++++++++++++++++++
SIN USO DE HERRAMIENTAS
++++++++++++++++++++++++++++++
Has pasado la evaluación - devolviendo respuesta
Herramienta llamada: record_user_details
Push: Registrando interés de Carlos, de la empresa Pretensa, telefono no proporcionado y notas not provided
FIN HERRAMIENTA
++++++++++++++++++++++++++++++
SIN USO DE HERRAMIENTAS
++++++++++++++++++++++++++++++
Has pasado la evaluación - devolviendo respuesta
